<a href="https://colab.research.google.com/github/riemannruiz/riemannruiz.github.io/blob/master/P0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Índices de similitud**
En este ejercicio tiene como objetivo explorar los índices de similitud usando diferentes tipos de datos. Para este ejemplo se utilizan un conjunto de datos generados por alumnos de la universidad, a quienes les fue solicitado la calificación de un conjunto de películas en una escala de 1 a 5 estrellas.

Cargar las librerías necesarias para ejecutar el ejercicio.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm # metricas de similitud
import scipy.spatial.distance as sc # metricas de distancia

Si el ejercicio está ejecutandose en la plataforma **Google Colab**, se debe de cargar los datos a la plataforma para poder acceder a los mismos. La carga de los datos se realizá con el código siguiente.

Si el ejercicio se está ejecutando de forma local se debe omitir el código siguiente.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Leer e importar los datos mediante la paquetería pandas y los almacenamos en un DataFrame.

In [ ]:
file_path = 'Test de películas.xlsx'
data = pd.read_excel(file_path,encoding='latin_1',index_col=0)
data.head()

Seleccionar las columnas que contienen los datos necesarios para limpiar los datos que serán usados.

In [ ]:
def seleccionar_columnas(x):
  csel = np.arange(9,246,3)
  usuarios1 = list(x.iloc[:,6])
  cnames1 = list(x.columns.values[csel])
  x = x[cnames1]
  x.index = usuarios1
  x[np.isnan(x)]=0
  return x

datan =  seleccionar_columnas(data)
datan.head()

## **Datos Binarios**

Cambiar la calificación de estrellas (multiestado ordinal) a un rango de **me gusta** y **no me gusta**. Esta operación es necesaria porque los índices de similitud a probar son para datos binarios.

In [ ]:
cnames = list(datan.columns.values)
fnames = np.array(datan.index)
for col in cnames:
    datan[col]=np.where(datan[col]>3,1,0)
datan.head()

Obtener la matriz de confusión (matriz de contingencia) y los indices de similitud.

In [ ]:
cf_m = skm.confusion_matrix(datan.iloc[0,:],datan.iloc[1,:])

sim_simple = skm.accuracy_score(datan.iloc[0,:],datan.iloc[1,:])
#sim_simple_new = (cf_m[0,0]+cf_m[1,1])/np.sum(cf_m)
print('Simple : %0.4f'%sim_simple)

sim_jac = skm.jaccard_score(datan.iloc[0,:],datan.iloc[1,:])
#sim_jac = (cf_m[1,1])/(np.sum(cf_m)-cf_m[0,0])
print('Jaccard: %0.4f'%sim_jac)

Dependiendo de la librería usado, algunas pueden calcular las distancias en lugar de los índices de similitud directamente.

In [ ]:
#%% Calculo de las distancias
d1 = sc.matching(datan.iloc[0,:],datan.iloc[1,:])
print('Simple : %0.4f'%d1)
d2 = sc.jaccard(datan.iloc[0,:],datan.iloc[1,:])
print('Jaccard: %0.4f'%d2)

Calcular los índices de similitud puede ser complicado debido a que la cantidad de operaciones está en función de la cantidad de datos que se tienen en el conjunto de datos.

In [ ]:
#%% Calcular todas las combinaciones posibles
D1 = sc.pdist(datan,'matching')
D1 = sc.squareform(D1)

D2 = sc.pdist(datan,'jaccard')
D2 = sc.squareform(D2)

In [ ]:
# Visualizar los índices calculados
pd.DataFrame(D1)

Tener todos los índices de similitud en una tabla puede confundir, y el uso dependerá de la finalidad de estas métricas. Por el momento nos enfocaremos a seleccionar un usuario en específico.

In [ ]:
#%% Seleccionar un usuario y determinar los demás usuarios más parecidos
user = 1
D_user = D1[user]
D_user_sort = np.sort(D_user)
indx_user = np.argsort(D_user)

Recomendación de películas versión 1.

In [ ]:
User = datan.loc[fnames[user]]
User_sim = datan.loc[fnames[indx_user[1]]]

indx_recomen = (User_sim ==1)&(User==0)
recomend1 = list(User.index[indx_recomen])
recomend1

Recomendación de películas versión 2.

In [ ]:
User = datan.loc[fnames[user]]
User_sim = np.mean(datan.loc[fnames[indx_user[1:6]]],axis=0)
User_sim[User_sim<=0.5] = 0
User_sim[User_sim>0.5] = 1

indx_recomen = (User_sim ==1)&(User==0)
recomend2 = list(User.index[indx_recomen])
recomend2

## **Datos Multiestado**

Seleccionar los datos sin necesidad de convertir la escala de estrellas a datos binarios.

In [ ]:
datan =  seleccionar_columnas(data)
datan.head()

Los índices de similitud más comunes están adaptados para funcionar con datos multiestado.

In [ ]:
cf_m = skm.confusion_matrix(datan.iloc[0,:],datan.iloc[1,:])
sim_simple = skm.accuracy_score(datan.iloc[0,:],datan.iloc[1,:])
sim_jac = skm.jaccard_score(datan.iloc[0,:],datan.iloc[1,:],average='weighted')

Para los índices de similitud que no están adaptados, pueden convertirse los datos multiestado a datos binarios usando variables auxiliares.

In [ ]:
#%% Generar variables dummies 
dum1 = pd.get_dummies(datan[cnames[0]])
#dum1 = pd.get_dummies(datan[cnames[0]],prefix=cnames[0])
dum1

Obteniendo todas las variables auxiliares de todas las películas, hace que el conjunto de datos crezca considerablemente.

In [ ]:
datan_dum = pd.get_dummies(datan[cnames[0]],prefix=cnames[0])
for col in cnames[1:]:
    tmp = pd.get_dummies(datan[col],prefix=col)
    datan_dum = datan_dum.join(tmp)
del tmp
datan_dum.head()

## **Datos Cuantitativos**

Los datos cuantitativos pueden procesarse directamente por medio de otros índices de similitud diseñados para este tipo de datos.

In [ ]:
datan =  seleccionar_columnas(data)
datan.head()

La distancia Euclideana puede considerarse como una métrica de similitud porque mide la distancia geométrica entre dos puntos en un espacio multidimensional.

In [ ]:
#%% Distancia Euclideana
D1 = sc.pdist(datan,'euclidean')
D1 = sc.squareform(D1)
pd.DataFrame(D1)

La distancia coseno mide el ángulo entre dos punto en un espacio multidimensional.

In [ ]:
#%% Distancia coseno
D2 = sc.pdist(datan,'cosine')
D2 = sc.squareform(D2)
pd.DataFrame(D2)

El índice de correlación tambien puede ser usada para medir la similitud entre muestras en un conjunto de datos.

In [ ]:
#%% Indice correlación
D3 = sc.pdist(datan,'correlation')
D3 = sc.squareform(D3)
pd.DataFrame(D3)